# Shear Mode Device Example

## Boilerplate

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import bokeh.palettes as palettes

from pathlib import Path
from comsol_mesh import *

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import Column, gridplot, Row

palette = palettes.Category10[10]

output_notebook()

Loading BokehJS ...

## Example: Importing a COMSOL mesh object

In [5]:
path = Path('data/shear_device.mphtxt')
comsol_objs = COMSOLObjects.from_file(path)
cobj = comsol_objs[0]

mesh = Mesh.from_comsol_obj(cobj)

In [6]:
points = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])
tet_indices = np.array([[0, 1, 2, 3]])
mesh = Mesh(points, tet_indices)

In [7]:
# Test cube integration
points = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1],
])
tet_indices = np.array([
    [0, 1, 2, 4],
    [2, 4, 5, 6],
    [1, 2, 4, 5],
    [1, 2, 3, 7],
    [2, 5, 6, 7],
    [1, 2, 5, 7]
])

cube_mesh = Mesh(points, tet_indices)
cube_mesh.volume()

0.9999999999999999

## Example: Importing a field

In [8]:
path = Path('data/shear_device_eigenmodes.csv')
cfield = COMSOLField.from_file(path)

In [9]:
cfield.values.shape

(2823, 30)

In [10]:
path = Path('data/shear_device_eigenmodes.csv')
cemodes = COMSOLEigenmodes.from_file(path)

In [11]:
cemodes.values.shape

(2823, 10, 3)

In [12]:
path = Path('data/shear_device.mphtxt')
comsol_objs = COMSOLObjects.from_file(path)
cobj = comsol_objs[0]

mesh = Mesh.from_comsol_obj(cobj)
mesh

Mesh(n_points=2823, n_tetrahedra=13851)

In [13]:
modes_field = Field.from_comsol_field(mesh, cemodes)
modes_field

Field(mesh=Mesh(n_points=2823, n_tetrahedra=13851), field_shape=(10, 3))

In [14]:
show(plot_points(mesh.points))

In [15]:
surfaces = surfaces_from_comsol_obj(mesh, cobj)
top_surface = surfaces[-1]
show(plot_points(top_surface.points()))

In [16]:
field = Field(mesh, values=np.ones(mesh.n_points).reshape((-1, 1)))

In [17]:
modes_field.eval_surface(top_surface, 0, np.array([1/3, 1/3, 1/3])).shape

(10, 3)

In [18]:
points, values = top_surface.random_value_sample(modes_field, 1000)

In [19]:
points = top_surface.random_point_sample(1000)
show(plot_points(points))

In [46]:
freq_shifts = 0.5 * np.linalg.norm(values, axis=-1)

p = figure()
p.scatter(freq_shifts[:, 5], freq_shifts[:, 6])
show(p)

